In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset
dataset = pd.read_csv('G:/Udacity/Datasets/creditcard.csv')
dataset.head()

#Feature-scaling on the Amount attribute.
from sklearn.preprocessing import StandardScaler
dataset['Amount'] = StandardScaler().fit_transform(dataset['Amount'].values.reshape(-1, 1))

In [3]:
dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [4]:
X = dataset.iloc[:, 1:30].values
y = dataset.iloc[:, 30].values

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 29))

# Adding the second hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

#Defining our metric for cost reduction

def as_keras_metric(method):
    import functools
    from keras import backend as K
    import tensorflow as tf
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

@as_keras_metric
def auc_pr(y_true, y_pred, curve='PR'):
    return tf.metrics.auc(y_true, y_pred, curve=curve, summation_method='careful_interpolation')

# precision = as_keras_metric(tf.metrics.precision)
# recall = as_keras_metric(tf.metrics.recall)

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [auc_pr])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 100, epochs = 30)


Epoch 1/30
227845/227845 [==============================] - 7s 31us/step - loss: 0.0252 - auc_pr: 0.0165
Epoch 2/30
227845/227845 [==============================] - 7s 30us/step - loss: 0.0032 - auc_pr: 0.2920
Epoch 3/30
227845/227845 [==============================] - 8s 37us/step - loss: 0.0029 - auc_pr: 0.4592
Epoch 4/30
227845/227845 [==============================] - 8s 36us/step - loss: 0.0028 - auc_pr: 0.5477
Epoch 5/30
227845/227845 [==============================] - 8s 36us/step - loss: 0.0027 - auc_pr: 0.5918
Epoch 6/30
227845/227845 [==============================] - 8s 34us/step - loss: 0.0026 - auc_pr: 0.6167
Epoch 7/30
227845/227845 [==============================] - 8s 33us/step - loss: 0.0025 - auc_pr: 0.6372
Epoch 8/30
227845/227845 [==============================] - 7s 32us/step - loss: 0.0024 - auc_pr: 0.6579
Epoch 9/30
227845/227845 [==============================] - 7s 33us/step - loss: 0.0024 - auc_pr: 0.6701
Epoch 10/30
227845/227845 [============================

In [10]:
# Part 3 - Making predictions and evaluating the model
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
from sklearn.metrics import recall_score
print(recall_score(y_test, y_pred, average='macro'))

0.8909923818044385
